# Azure OpenAI LangChain Quickstart

In this quickstart you will create a simple LangChain App and learn how to log it and get feedback on an LLM response using both an embedding and chat completion model from Azure OpenAI.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/azure_openai_langchain.ipynb)

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [ ]:
#! pip install trulens-eval==0.25.0 llama-index==0.10.17 langchain==0.1.11 chromadb==0.4.24 langchainhub bs4==0.0.2 langchain-openai==0.0.8 ipytree==0.2.2

### Add API keys
For this quickstart, you will need a larger set of information from Azure OpenAI compared to typical OpenAI usage. These can be retrieved from https://oai.azure.com/ . Deployment name below is also found on the oai azure page.

In [ ]:
# Check your https://oai.azure.com dashboard to retrieve params:

import os
os.environ["AZURE_OPENAI_API_KEY"] = "..." # azure
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<your endpoint here>.openai.azure.com/" # azure
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview" # may need updating
os.environ["OPENAI_API_TYPE"] = "azure"

### Import from TruLens

In [ ]:
# Imports main tools:
from trulens_eval import TruChain, Feedback, Tru
tru = Tru()
tru.reset_database()

### Create Simple LLM Application

This example uses LangChain and is set to use Azure OpenAI LLM & Embedding Models

In [ ]:
import os
import bs4 

# Langchain imports
from langchain import hub
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

# Imports Azure LLM & Embedding from LangChain
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

import logging
import sys


### Define the LLM & Embedding Model

In [ ]:
# get model from Azure
llm = AzureChatOpenAI(
    model="gpt-35-turbo",
    deployment_name="<your azure deployment name>", # Replace this with your azure deployment name
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbeddings(
    azure_deployment="soc-text",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

### Load Doc & Split & Create Vectorstore
#### 1. Load the Document

In [ ]:
# Load a sample document
loader = WebBaseLoader(
    web_paths=("http://paulgraham.com/worked.html",),
)
docs = loader.load()

#### 2. Split the Document

In [ ]:
# Define a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# Apply text splitter to docs
splits = text_splitter.split_documents(docs)


#### 3. Create a Vectorstore

In [ ]:
# Create a vectorstore from splits
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embed_model
)

### Create a RAG Chain

In [ ]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = llm

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Send your first request

In [ ]:
query = "What is most interesting about this essay?"
answer = rag_chain.invoke(query)

print("query was:", query)
print("answer was:", answer)

## Initialize Feedback Function(s)

In [ ]:
from trulens_eval.feedback.provider import AzureOpenAI
from trulens_eval.feedback import Groundedness
import numpy as np


# Initialize AzureOpenAI-based feedback function collection class:
azopenai = AzureOpenAI(
     # Replace this with your azure deployment name
    deployment_name="<your azure deployment name>")


# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)


# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    azopenai.relevance, 
    name = "Answer Relevance"
    ).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(
    azopenai.qs_relevance_with_cot_reasons, name = "Context Relevance").on_input().on(context).aggregate(np.mean)

# groundedness of output on the context
groundedness = Groundedness(groundedness_provider=azopenai)

f_groundedness = (Feedback(groundedness.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(context.collect())
    .on_output()
    .aggregate(groundedness.grounded_statements_aggregator)
)

## Custom functions can also use the Azure provider

In [ ]:
from typing import Tuple, Dict
from trulens_eval.feedback import prompts

from trulens_eval.utils.generated import re_0_10_rating

class Custom_AzureOpenAI(AzureOpenAI):
    def style_check_professional(self, response: str) -> float:
        """
        Custom feedback function to grade the professional style of the resposne, extending AzureOpenAI provider.

        Args:
            response (str): text to be graded for professional style.

        Returns:
            float: A value between 0 and 1. 0 being "not professional" and 1 being "professional".
        """
        professional_prompt = str.format("Please rate the professionalism of the following text on a scale from 0 to 10, where 0 is not at all professional and 10 is extremely professional: \n\n{}", response)
        return self.generate_score(system_prompt=professional_prompt)
    
    def qs_relevance_with_cot_reasons_extreme(self, question: str, statement: str) -> Tuple[float, Dict]:
        """
        Tweaked version of question statement relevance, extending AzureOpenAI provider.
        A function that completes a template to check the relevance of the statement to the question.
        Scoring guidelines for scores 5-8 are removed to push the LLM to more extreme scores.
        Also uses chain of thought methodology and emits the reasons.

        Args:
            question (str): A question being asked. 
            statement (str): A statement to the question.

        Returns:
            float: A value between 0 and 1. 0 being "not relevant" and 1 being "relevant".
        """

        system_prompt = str.format(prompts.CONTEXT_RELEVANCE, question = question, statement = statement)

        # remove scoring guidelines around middle scores
        system_prompt = system_prompt.replace(
    "- STATEMENT that is RELEVANT to most of the QUESTION should get a score of 5, 6, 7 or 8. Higher score indicates more RELEVANCE.\n\n", "")
        
        system_prompt = system_prompt.replace(
            "RELEVANCE:", prompts.COT_REASONS_TEMPLATE
        )

        return self.generate_score_and_reasons(system_prompt)
    
# Add your Azure deployment name
custom_azopenai = Custom_AzureOpenAI(deployment_name="<your azure deployment name>")
    
# Question/statement relevance between question and each context chunk.
f_qs_relevance_extreme = (
    Feedback(custom_azopenai.qs_relevance_with_cot_reasons_extreme, name = "Context Relevance - Extreme")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

f_style_check = (
    Feedback(custom_azopenai.style_check_professional, name = "Professional Style")
    .on_output()
)

## Instrument chain for logging with TruLens

In [ ]:
tru_query_engine_recorder = TruChain(rag_chain,
    llm=azopenai,
    app_id='LangChain_App1_AzureOpenAI',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance, f_qs_relevance_extreme, f_style_check])

In [ ]:
query = "What is most interesting about this essay?"
with tru_query_engine_recorder as recording:
    answer = rag_chain.invoke(query)
    print("query was:", query)
    print("answer was:", answer)

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=['LangChain_App1_AzureOpenAI']) # pass an empty list of app_ids to get all

records

In [ ]:
tru.get_leaderboard(app_ids=['LangChain_App1_AzureOpenAI'])